In [1]:
import sqlite3
conn = sqlite3.connect('inverted-index.db')

In [2]:
# Create table
c = conn.cursor()

c.execute('''
    CREATE TABLE IndexWord (
        word TEXT PRIMARY KEY
    );
''')

c.execute('''
    CREATE TABLE Posting (
        word TEXT NOT NULL,
        documentName TEXT NOT NULL,
        frequency INTEGER NOT NULL,
        indexes TEXT NOT NULL,
        PRIMARY KEY(word, documentName),
        FOREIGN KEY (word) REFERENCES IndexWord(word)
    );
''')

# Save (commit) the changes
conn.commit()

# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
conn.close()

In [3]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

stop_words_slovene = set(stopwords.words("slovene")).union(set(
        ["ter","nov","novo", "nova","zato","še", "zaradi", "a", "ali", "april", "avgust", "b", "bi", "bil", "bila", "bile", "bili", "bilo", "biti",
         "blizu", "bo", "bodo", "bojo", "bolj", "bom", "bomo", "boste", "bova", "boš", "brez", "c", "cel", "cela",
         "celi", "celo", "d", "da", "daleč", "dan", "danes", "datum", "december", "deset", "deseta", "deseti", "deseto",
         "devet", "deveta", "deveti", "deveto", "do", "dober", "dobra", "dobri", "dobro", "dokler", "dol", "dolg",
         "dolga", "dolgi", "dovolj", "drug", "druga", "drugi", "drugo", "dva", "dve", "e", "eden", "en", "ena", "ene",
         "eni", "enkrat", "eno", "etc.", "f", "februar", "g", "g.", "ga", "ga.", "gor", "gospa", "gospod", "h", "halo",
         "i", "idr.", "ii", "iii", "in", "iv", "ix", "iz", "j", "januar", "jaz", "je", "ji", "jih", "jim", "jo",
         "julij", "junij", "jutri", "k", "kadarkoli", "kaj", "kajti", "kako", "kakor", "kamor", "kamorkoli", "kar",
         "karkoli", "katerikoli", "kdaj", "kdo", "kdorkoli", "ker", "ki", "kje", "kjer", "kjerkoli", "ko", "koder",
         "koderkoli", "koga", "komu", "kot", "kratek", "kratka", "kratke", "kratki", "l", "lahka", "lahke", "lahki",
         "lahko", "le", "lep", "lepa", "lepe", "lepi", "lepo", "leto", "m", "maj", "majhen", "majhna", "majhni",
         "malce", "malo", "manj", "marec", "me", "med", "medtem", "mene", "mesec", "mi", "midva", "midve", "mnogo",
         "moj", "moja", "moje", "mora", "morajo", "moram", "moramo", "morate", "moraš", "morem", "mu", "n", "na", "nad",
         "naj", "najina", "najino", "najmanj", "naju", "največ", "nam", "narobe", "nas", "nato", "nazaj", "naš", "naša",
         "naše", "ne", "nedavno", "nedelja", "nek", "neka", "nekaj", "nekatere", "nekateri", "nekatero", "nekdo",
         "neke", "nekega", "neki", "nekje", "neko", "nekoga", "nekoč", "ni", "nikamor", "nikdar", "nikjer", "nikoli",
         "nič", "nje", "njega", "njegov", "njegova", "njegovo", "njej", "njemu", "njen", "njena", "njeno", "nji",
         "njih", "njihov", "njihova", "njihovo", "njiju", "njim", "njo", "njun", "njuna", "njuno", "no", "nocoj",
         "november", "npr.", "o", "ob", "oba", "obe", "oboje", "od", "odprt", "odprta", "odprti", "okoli", "oktober",
         "on", "onadva", "one", "oni", "onidve", "osem", "osma", "osmi", "osmo", "oz.", "p", "pa", "pet", "peta",
         "petek", "peti", "peto", "po", "pod", "pogosto", "poleg", "poln", "polna", "polni", "polno", "ponavadi",
         "ponedeljek", "ponovno", "potem", "povsod", "pozdravljen", "pozdravljeni", "prav", "prava", "prave", "pravi",
         "pravo", "prazen", "prazna", "prazno", "prbl.", "precej", "pred", "prej", "preko", "pri", "pribl.",
         "približno", "primer", "pripravljen", "pripravljena", "pripravljeni", "proti", "prva", "prvi", "prvo", "r",
         "ravno", "redko", "res", "reč", "s", "saj", "sam", "sama", "same", "sami", "samo", "se", "sebe", "sebi",
         "sedaj", "sedem", "sedma", "sedmi", "sedmo", "sem", "september", "seveda", "si", "sicer", "skoraj", "skozi",
         "slab", "smo", "so", "sobota", "spet", "sreda", "srednja", "srednji", "sta", "ste", "stran", "stvar", "sva",
         "t", "ta", "tak", "taka", "take", "taki", "tako", "takoj", "tam", "te", "tebe", "tebi", "tega", "težak",
         "težka", "težki", "težko", "ti", "tista", "tiste", "tisti", "tisto", "tj.", "tja", "to", "toda", "torek",
         "tretja", "tretje", "tretji", "tri", "tu", "tudi", "tukaj", "tvoj", "tvoja", "tvoje", "u", "v", "vaju", "vam",
         "vas", "vaš", "vaša", "vaše", "ve", "vedno", "velik", "velika", "veliki", "veliko", "vendar", "ves", "več",
         "vi", "vidva", "vii", "viii", "visok", "visoka", "visoke", "visoki", "vsa", "vsaj", "vsak", "vsaka", "vsakdo",
         "vsake", "vsaki", "vsakomur", "vse", "vsega", "vsi", "vso", "včasih", "včeraj", "x", "z", "za", "zadaj",
         "zadnji", "zakaj", "zaprta", "zaprti", "zaprto", "zdaj", "zelo", "zunaj", "č", "če", "često", "četrta",
         "četrtek", "četrti", "četrto", "čez", "čigav", "š", "šest", "šesta", "šesti", "šesto", "štiri", "ž", "že",
         "svoj", "jesti", "imeti","\u0161e", "iti", "kak", "www", "km", "eur", "pač", "del", "kljub", "šele", "prek",
         "preko", "znova", "morda","kateri","katero","katera", "ampak", "lahek", "lahka", "lahko", "morati", "torej",
         "(", ")", "--", ";", ".", ",", "/", "-", "!", "?", "'", ":"]))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\miham\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
from bs4 import BeautifulSoup
import os
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize.util import string_span_tokenize
import tokenizations
import sqlite3

conn = sqlite3.connect('inverted-index.db')
cur = conn.cursor()

path_to_file = "C:/Users/miham/Documents/Faks/IEPS/PA3/PA3-data/evem.gov.si/evem.gov.si.4.html"
root = "C:/Users/miham/Documents/Faks/IEPS/PA3/PA3-data"
stop_words_english = stopwords.words('english')

#tokenize, insert into database test
"""
with open(path_to_file, encoding="utf8") as fp:
    soup = BeautifulSoup(fp, 'html.parser')
    
    for script in soup(["script", "style"]):
        script.extract()    # remove script tags, style tags
            
    # get text
    text = soup.get_text()
    text = text.lower()
    tokens = word_tokenize(text)
    tokens = [t for t in tokens if not t in stop_words_slovene]
    tokens = [t for t in tokens if not t in stop_words_english]
    
    spans = []
    ix = 0
    for token in tokens:
        ix = text.find(token, ix)
        spans.append(ix)
        ix += len(token)
    
    #print(tokens)
    #print(spans)

    rev_ix = {}
    for i in range(len(tokens)):
        if tokens[i] not in rev_ix:
            rev_ix[tokens[i]] = [1, [spans[i]]]
        else:
            rev_ix[tokens[i]][0] += 1
            rev_ix[tokens[i]][1].append(spans[i])
    
    addr = 'evem.gov.si/evem.gov.si.4.html'
    for key in rev_ix:
        cur.execute('''SELECT word FROM IndexWord WHERE word =?''', (key,))
        exists = cur.fetchall()
        data = rev_ix[key]
        f = data[0]
        indices = ','.join(str(e) for e in data[1])
        
        if not exists:
            cur.execute('''INSERT INTO IndexWord VALUES (?)''', (key,))
            cur.execute('''INSERT INTO Posting VALUES (?,?,?,?)''',(key, addr, f, indices))
        else:
            cur.execute('''INSERT INTO Posting VALUES (?,?,?,?)''',(key, addr, f, indices))
            
        conn.commit()
"""
for subdir, dirs, files in os.walk(root):
    for file in files:
        path = os.path.join(subdir, file)
        domain = os.path.basename(os.path.normpath(subdir))
        site = file
        addr = domain + '/' + site
 
        with open(path, encoding='utf8') as fp:
            soup = BeautifulSoup(fp, 'html.parser')
    
            for script in soup(["script", "style"]):
                script.extract()    # remove script tags, style tags

            # get text, tokenize text, and remove stop words
            text = soup.get_text()
            text = text.lower()
            tokens = word_tokenize(text)
            tokens = [t for t in tokens if not t in stop_words_slovene]
            tokens = [t for t in tokens if not t in stop_words_english]
            
            # find spans of tokens in original text 
            spans = []
            ix = 0
            for token in tokens:
                ix = text.find(token, ix)
                spans.append(ix)
                ix += len(token)
            
            # build local reverse index for given document
            rev_ix = {}
            for i in range(len(tokens)):
                if tokens[i] not in rev_ix:
                    rev_ix[tokens[i]] = [1, [spans[i]]]
                else:
                    rev_ix[tokens[i]][0] += 1
                    rev_ix[tokens[i]][1].append(spans[i])
            
            # write value to database
            for key in rev_ix:
                cur.execute('''SELECT word FROM IndexWord WHERE word =?''', (key,))
                exists = cur.fetchall()
                data = rev_ix[key]
                f = data[0]
                indices = ','.join(str(e) for e in data[1])
                
                # check if token exists in IndexWords
                if not exists:
                    cur.execute('''INSERT INTO IndexWord VALUES (?)''', (key,)) 
                    cur.execute('''INSERT INTO Posting VALUES (?,?,?,?)''',(key, addr, f, indices))
                else:
                    cur.execute('''INSERT INTO Posting VALUES (?,?,?,?)''',(key, addr, f, indices))

                conn.commit()

print("DONE!")
         
            


DONE!
